# **1. Import Libraries**

In [16]:
import pandas as pd
import numpy as np

In [8]:
# Langkah 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# **2. Baca Dataset Pelanggan Yang Kotor**

In [ ]:
# Load dataset
file_path = "/content/drive/MyDrive/Colab Notebooks/Messy_Employee.csv"# ganti sesuai path di Colab
df = pd.read_csv(file_path)

In [ ]:
# Tampilkan data
print(df)

     Employee_ID First_Name Last_Name   Age   Department_Region    Status  \
0        EMP1000        Bob     Davis  25.0   DevOps-California    Active   
1        EMP1001        Bob     Brown   NaN       Finance-Texas    Active   
2        EMP1002      Alice     Jones   NaN        Admin-Nevada   Pending   
3        EMP1003        Eva     Davis  25.0        Admin-Nevada  Inactive   
4        EMP1004      Frank  Williams  25.0  Cloud Tech-Florida    Active   
...          ...        ...       ...   ...                 ...       ...   
1015     EMP2015      David    Miller  30.0       HR-California    Active   
1016     EMP2016      David   Johnson  30.0    Cloud Tech-Texas  Inactive   
1017     EMP2017    Charlie  Williams  40.0    Finance-New York    Active   
1018     EMP2018      Alice    Garcia  30.0          HR-Florida  Inactive   
1019     EMP2019      Heidi     Jones  30.0     DevOps-Illinois   Pending   

       Join_Date     Salary                         Email       Phone  \
0 

# **3. Menampilkan Struktur Variable Data Employee**

In [9]:
print("Struktur Data Pelanggan:")
print(df.info())

Struktur Data Pelanggan:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Employee_ID        1020 non-null   object 
 1   First_Name         1020 non-null   object 
 2   Last_Name          1020 non-null   object 
 3   Age                809 non-null    float64
 4   Department_Region  1020 non-null   object 
 5   Status             1020 non-null   object 
 6   Join_Date          1020 non-null   object 
 7   Salary             996 non-null    float64
 8   Email              1020 non-null   object 
 9   Phone              1020 non-null   int64  
 10  Performance_Score  1020 non-null   object 
 11  Remote_Work        1020 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 88.8+ KB
None


In [10]:
# Cek struktur awal
print("Info Data Awal:")
print(df.info())
print("\nJumlah Missing Values per Kolom:")
print(df.isnull().sum())

Info Data Awal:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Employee_ID        1020 non-null   object 
 1   First_Name         1020 non-null   object 
 2   Last_Name          1020 non-null   object 
 3   Age                809 non-null    float64
 4   Department_Region  1020 non-null   object 
 5   Status             1020 non-null   object 
 6   Join_Date          1020 non-null   object 
 7   Salary             996 non-null    float64
 8   Email              1020 non-null   object 
 9   Phone              1020 non-null   int64  
 10  Performance_Score  1020 non-null   object 
 11  Remote_Work        1020 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 88.8+ KB
None

Jumlah Missing Values per Kolom:
Employee_ID            0
First_Name             0
Last_Name              0
Age                

In [11]:
# Tampilkan nama kolom dan tipe data kolom
column_types = df.dtypes
print(column_types)

Employee_ID           object
First_Name            object
Last_Name             object
Age                  float64
Department_Region     object
Status                object
Join_Date             object
Salary               float64
Email                 object
Phone                  int64
Performance_Score     object
Remote_Work             bool
dtype: object


# **4. Data Cleansing-Standarisan Employee_Id**

In [12]:
def format_employee_id(kode):
    if isinstance(kode, str) and kode.startswith("EMP-"):
        num = kode.split('-')[-1]
        return "EMP-" + num.zfill(4)
    return kode

df['Employee_ID'] = df['Employee_ID'].astype(str).apply(format_employee_id)

# **5. Data Cleansing-Mengabungkan Nama Lengkap**

In [13]:
df['Full_Name'] = (df['First_Name'].astype(str).str.strip().str.title() + " " +
                   df['Last_Name'].astype(str).str.strip().str.title())

# **6. Data Cleansing-Standarisa Join-Date**

In [14]:
df['Join_Date'] = pd.to_datetime(df['Join_Date'], errors='coerce')

# **7. Data Cleansing-Phone Number**

In [17]:
df['Phone'] = df['Phone'].astype(str).str.replace('-', '', regex=True)
df.loc[~df['Phone'].str.isnumeric(), 'Phone'] = np.nan
df.loc[~df['Phone'].str.len().between(10,13), 'Phone'] = np.nan

# **8. Data Cleansing-Pecah Department dan Region**

In [18]:
df[['Department','Region']] = df['Department_Region'].str.split('-', n=1, expand=True)

# **9. Data Cleansing Normalisasi Performance Score**

In [19]:
score_map = {"Poor":1,"Average":2,"Good":3,"Excellent":4}
df['Performance_Score_Num'] = df['Performance_Score'].map(score_map)

# **10. Mengisi Missing Values**

In [20]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Salary'] = df['Salary'].fillna(df['Salary'].mean())

# **11. Deduplikasi**

In [21]:
# Urutkan agar Employee_ID lebih kecil dipertahankan
df_sorted = df.sort_values(by='Employee_ID')

In [22]:
# Drop duplikasi berdasarkan Full_Name + Email + Join_Date
df = df_sorted.drop_duplicates(subset=['Full_Name','Email','Join_Date'], keep='first')
df.reset_index(drop=True, inplace=True)

print(f"Jumlah data setelah deduplikasi: {len(df)}")


Jumlah data setelah deduplikasi: 1014


# **12. Data Enrichment**

In [23]:
# Lama kerja (tahun)
df['Seniority_Years'] = (pd.to_datetime("today") - df['Join_Date']).dt.days // 365

# Kelompok usia
bins = [0,25,35,45,55,100]
labels = ["<25","25-34","35-44","45-54","55+"]
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels)

# Level gaji (Low, Medium, High)
df['Salary_Level'] = pd.qcut(df['Salary'], 3, labels=["Low","Medium","High"])

# Tampilkan hasil enrichment
df[['Employee_ID','Full_Name','Age','Age_Group','Salary','Salary_Level','Join_Date','Seniority_Years']].head(10)

/tmp/ipython-input-3524715003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Seniority_Years'] = (pd.to_datetime("today") - df['Join_Date']).dt.days // 365
/tmp/ipython-input-3524715003.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels)
/tmp/ipython-input-3524715003.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,Employee_ID,Full_Name,Age,Age_Group,Salary,Salary_Level,Join_Date,Seniority_Years
0,EMP1000,Bob Davis,25.0,<25,59767.65,Low,2021-04-02,4
1,EMP1001,Bob Brown,30.0,25-34,65304.66,Low,2020-07-10,5
2,EMP1002,Alice Jones,30.0,25-34,88145.90,Medium,2023-12-07,1
3,EMP1003,Eva Davis,25.0,<25,69450.99,Low,2021-11-27,3
4,EMP1004,Frank Williams,25.0,<25,109324.61,High,2022-01-05,3
5,EMP1005,Alice Garcia,40.0,35-44,88642.84,Medium,2020-06-10,5
6,EMP1006,Frank Jones,30.0,25-34,96288.43,High,2020-04-03,5
7,EMP1007,Bob Jones,30.0,25-34,94497.91,Medium,2022-07-17,3
8,EMP1008,Frank Davis,35.0,25-34,115565.82,High,2023-12-08,1
9,EMP1009,Charlie Johnson,30.0,25-34,76561.88,Medium,2022-08-04,3
